In [15]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [16]:
class State(TypedDict):
    query: str
    category: str
    sentiment: str
    response: str



In [ ]:
def categorize(state: State) -> State:
    """Categorize the customer query into Technical, Billing, or General."""
    prompt = ChatPromptTemplate.from_template(
        "Categorize the following customer query into one of these categories: "
        "Technical, Billing, General. Query: {query}"
    )
    chain = prompt | ChatGroq(temperature=0, model="llama-3.3-70b-versatile", max_tokens=4000,api_key='')
    category = chain.invoke({"query": state["query"]}).content
    return {"category": category}

In [ ]:
def analyze_sentiment(state: State) -> State:
    """Analyze the sentiment of the customer query as Positive, Neutral, or Negative."""
    prompt = ChatPromptTemplate.from_template(
        "Analyze the sentiment of the following customer query. "
        "Respond with either 'Positive', 'Neutral', or 'Negative'. Query: {query}"
    )
    chain = prompt | ChatGroq(temperature=0, model="llama-3.3-70b-versatile", max_tokens=4000,api_key='')
    sentiment = chain.invoke({"query": state["query"]}).content
    return {"sentiment": sentiment}

In [ ]:
def handle_technical(state: State) -> State:
    """Provide a technical support response to the query."""
    prompt = ChatPromptTemplate.from_template(
        "Provide a technical support response to the following query: {query}"
    )
    chain = prompt | ChatGroq(temperature=0, model="llama-3.3-70b-versatile", max_tokens=4000,api_key='')
    response = chain.invoke({"query": state["query"]}).content
    return {"response": response}

In [ ]:
def handle_billing(state: State) -> State:
    """Provide a billing support response to the query."""
    prompt = ChatPromptTemplate.from_template(
        "Provide a billing support response to the following query: {query}"
    )
    chain = prompt | ChatGroq(temperature=0, model="llama-3.3-70b-versatile", max_tokens=4000,api_key='')
    response = chain.invoke({"query": state["query"]}).content
    return {"response": response}

In [ ]:
def handle_general(state: State) -> State:
    """Provide a general support response to the query."""
    prompt = ChatPromptTemplate.from_template(
        "Provide a general support response to the following query: {query}"
    )
    chain = prompt | ChatGroq(temperature=0, model="llama-3.3-70b-versatile", max_tokens=4000,api_key='')
    response = chain.invoke({"query": state["query"]}).content
    return {"response": response}

In [22]:
def escalate(state: State) -> State:
    """Escalate the query to a human agent due to negative sentiment."""
    return {"response": "This query has been escalated to a human agent due to its negative sentiment."}

def route_query(state: State) -> str:
    """Route the query based on its sentiment and category."""
    if state["sentiment"] == "Negative":
        return "escalate"
    elif state["category"] == "Technical":
        return "handle_technical"
    elif state["category"] == "Billing":
        return "handle_billing"
    else:
        return "handle_general"

In [23]:
workflow = StateGraph(State)

workflow.add_node("categorize", categorize)
workflow.add_node("analyze_sentiment", analyze_sentiment)
workflow.add_node("handle_technical", handle_technical)
workflow.add_node("handle_billing", handle_billing)
workflow.add_node("handle_general", handle_general)
workflow.add_node("escalate", escalate)

In [24]:
workflow.add_edge("categorize", "analyze_sentiment")

In [25]:
workflow.add_conditional_edges(
    "analyze_sentiment",
    route_query,
    {
        "handle_technical": "handle_technical",
        "handle_billing": "handle_billing",
        "handle_general": "handle_general",
        "escalate": "escalate"
    }
)

In [26]:
workflow.add_edge("handle_technical", END)
workflow.add_edge("handle_billing", END)
workflow.add_edge("handle_general", END)
workflow.add_edge("escalate", END)

# Set entry point
workflow.set_entry_point("categorize")

# Compile the graph
app = workflow.compile()

In [27]:
def run_customer_support(query: str) -> Dict[str, str]:
    """Process a customer query through the LangGraph workflow.
    
    Args:
        query (str): The customer's query
        
    Returns:
        Dict[str, str]: A dictionary containing the query's category, sentiment, and response
    """
    results = app.invoke({"query": query})
    return {
        "category": results["category"],
        "sentiment": results["sentiment"],
        "response": results["response"]
    }

In [28]:
query = "My internet connection keeps dropping. Can you help?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

Query: My internet connection keeps dropping. Can you help?
Category: The customer query "My internet connection keeps dropping. Can you help?" can be categorized as: Technical. 

This is because the query is related to a technical issue with the customer's internet connection, and they are seeking assistance to resolve the problem.
Sentiment: Negative.
Response: I'm so sorry to hear that your internet connection is dropping. That can be really frustrating. I'd be happy to help you troubleshoot the issue. 

To get started, can you please tell me a bit more about the problem you're experiencing? For example: 

1. How often does your internet connection drop?
2. Are you using a wired or wireless connection?
3. Have you made any recent changes to your network or devices?
4. Are you getting any error messages when the connection drops?

Additionally, you may want to try a few basic troubleshooting steps to see if they resolve the issue:

1. Restart your router and modem to see if that stab

In [29]:
query = "where can i find my receipt?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

Query: where can i find my receipt?
Category: The customer query "where can I find my receipt?" can be categorized as 'Billing'. This is because the query is related to a financial transaction and the customer is looking for a document that confirms their payment.
Sentiment: Neutral. The customer is simply asking for information and does not express any emotion or opinion, so the sentiment is neutral.
Response: If you're having trouble finding your receipt, don't worry, I'm here to help. There are a few possible places you can check, depending on how you made your purchase. 

1. **Email**: If you made an online purchase, check your email inbox for a confirmation email that may include your receipt. It's often sent to you immediately after the transaction is complete.
2. **Order History**: If you have an account with the retailer, you can log in and check your order history. Many websites allow you to view and print receipts for past orders.
3. **Store or Restaurant**: If you made a pur